# Lab | LangChain Med

## Objectives

- continue on with lesson 2' example, use different datasets to test what we did in class. Some datasets are suggested in the notebook but feel free to scout other datasets on HuggingFace or Kaggle.
- Find another model on Hugging Face and compare it.
- Modify the prompt to fit your selected dataset.

In [1]:
import numpy as np 
import pandas as pd

## Load the Dataset
As you can see the notebook is ready to work with three different Datasets. Just uncomment the lines of the Dataset you want to use. 

I selected Datasets with News. Two of them have just a brief decription of the news, but the other contains the full text. 

As we are working in a free and limited space, I limited the number of news to use with the variable MAX_NEWS. Feel free to pull more if you have memory available. 

The name of the field containing the text of the new is stored in the variable *DOCUMENT* and the metadata in *TOPIC*

In [8]:
# news = pd.read_csv('/kaggle/input/topic-labeled-news-dataset/labelled_newscatcher_dataset.csv', sep=';')
# MAX_NEWS = 1000
# DOCUMENT="title"
# TOPIC="topic"

#news = pd.read_csv('/kaggle/input/bbc-news/bbc_news.csv')
#MAX_NEWS = 1000
#DOCUMENT="description"
#TOPIC="title"

#news = pd.read_csv('/kaggle/input/mit-ai-news-published-till-2023/articles.csv')
#MAX_NEWS = 100
#DOCUMENT="Article Body"
#TOPIC="Article Header"

news = pd.read_csv(r'C:\Users\happy\Documents\ironhack\Week18\lab-langchain-med\datasets\Receipes from around the world.csv', encoding='latin1')
MAX_NEWS = 100
DOCUMENT="recipe_name"
TOPIC="cuisine"
# news = "PICK A DATASET" #Ideally pick one from the commented ones above

In [6]:
news.shape

(161, 8)

In [7]:
news

,recipe_name,cuisine,ingredients,cooking_time_minutes,prep_time_minutes,servings,calories_per_serving,dietary_restrictions
0,Spicy Kimchi Fried Rice,Korean,"['Cooked rice', 'Kimchi', 'Gochujang', 'Soy sa...",25,15,2.0,450.0,['vegetarian']
1,Classic Margherita Pizza,Italian,"['Pizza dough', 'Tomato sauce', 'Fresh mozzare...",20,30,4.0,350.0,['vegetarian']
2,Coconut Chickpea Curry,Indian,"['Chickpeas', 'Coconut milk', 'Onion', 'Ginger...",35,15,4.0,480.0,"['vegan', 'vegetarian', 'gluten-free', 'dairy-..."
3,Pad See Ew with Tofu,Thai,"['Wide rice noodles', 'Tofu', 'Chinese broccol...",20,10,2.0,520.0,['vegetarian']
4,Beef Bourguignon,French,"['Beef chuck', 'Red wine', 'Bacon', 'Onion', '...",150,30,6.0,680.0,['nan']
...,...,...,...,...,...,...,...,...
156,Thai Lime Grill,Thai,"['lime', 'basil', 'lemongrass', 'peanuts', 'ch...",90,10,2.0,229.0,['nan']
157,Korean Scallions Delight,Korean,"['garlic', 'sesame oil', 'soy sauce', 'gochuja...",86,7,6.0,437.0,['vegan']
158,Nigerian Cassava Feast,Nigerian,"['palm oil', 'okra', 'egusi', 'cassava', 'yams...",39,44,3.0,389.0,['nan']
159,American Corn Curry,American,"['bacon', 'beef', 'mustard', 'corn', 'cheddar']",58,30,1.0,749.0,['nan']


ChromaDB requires that the data has a unique identifier. We can make it with this statement, which will create a new column called **Id**.


In [9]:
news["id"] = news.index
news.head()

,recipe_name,cuisine,ingredients,cooking_time_minutes,prep_time_minutes,servings,calories_per_serving,dietary_restrictions,id
0,Spicy Kimchi Fried Rice,Korean,"['Cooked rice', 'Kimchi', 'Gochujang', 'Soy sa...",25,15,2.0,450.0,['vegetarian'],0
1,Classic Margherita Pizza,Italian,"['Pizza dough', 'Tomato sauce', 'Fresh mozzare...",20,30,4.0,350.0,['vegetarian'],1
2,Coconut Chickpea Curry,Indian,"['Chickpeas', 'Coconut milk', 'Onion', 'Ginger...",35,15,4.0,480.0,"['vegan', 'vegetarian', 'gluten-free', 'dairy-...",2
3,Pad See Ew with Tofu,Thai,"['Wide rice noodles', 'Tofu', 'Chinese broccol...",20,10,2.0,520.0,['vegetarian'],3
4,Beef Bourguignon,French,"['Beef chuck', 'Red wine', 'Bacon', 'Onion', '...",150,30,6.0,680.0,['nan'],4


In [10]:
#Because it is just a course we select a small portion of News.
subset_news = news.head(MAX_NEWS)

## Import and configure the Vector Database
I'm going to use ChromaDB, the most popular OpenSource embedding Database. 

First we need to import ChromaDB, and after that import the **Settings** class from **chromadb.config** module. This class allows us to change the setting for the ChromaDB system, and customize its behavior. 

In [11]:
!pip install chromadb

In [12]:
import chromadb
from chromadb.config import Settings

Now we need to create the seetings object calling the **Settings** function imported previously. We store the object in the variable **settings_chroma**.

Is necessary to inform two parameters 
* chroma_db_impl. Here we specify the database implementation and the format how store the data. I choose ***duckdb***, because his high-performace. It operate primarly in memory. And is fully compatible with SQL. The store format ***parquet*** is good for tabular data. With good compression rates and performance. 

* persist_directory: It just contains the directory where the data will be stored. Is possible work without a directory and the data will be stored in memory without persistece, but Kaggle dosn't support that. 

In [13]:
chroma_client = chromadb.PersistentClient(path="/path/to/persist/directory")

## Filling and Querying the ChromaDB Database
The Data in ChromaDB is stored in collections. If the collection exist we need to delete it. 

In the next lines, we are creating the collection by calling the ***create_collection*** function in the ***chroma_client*** created above.

In [14]:
collection_name = "news_collection"
if len(chroma_client.list_collections()) > 0 and collection_name in [chroma_client.list_collections()[0].name]:
        chroma_client.delete_collection(name=collection_name)

collection = chroma_client.create_collection(name=collection_name)
    

It's time to add the data to the collection. Using the function ***add*** we need to inform, at least ***documents***, ***metadatas*** and ***ids***. 
* In the **document** we store the big text, it's a different column in each Dataset. 
* In **metadatas**, we can informa a list of topics. 
* In **id** we need to inform an unique identificator for each row. It MUST be unique! I'm creating the ID using the range of MAX_NEWS. 


In [15]:

collection.add(
    documents=subset_news[DOCUMENT].tolist(),
    metadatas=[{TOPIC: topic} for topic in subset_news[TOPIC].tolist()],
    ids=[f"id{x}" for x in range(MAX_NEWS)],
)

In [16]:
results = collection.query(query_texts=["laptop"], n_results=10 )

print(results)

{'ids': [['id54', 'id2', 'id69', 'id77', 'id83', 'id81', 'id70', 'id3', 'id4', 'id47']], 'embeddings': None, 'documents': [['Nigerian Suya (Grilled Spicy Skewered Meat)', 'Coconut Chickpea Curry', 'Thai Panang Curry with Chicken', 'Nigerian Moi Moi (Steamed Bean Pudding)', 'Vietnamese Bun Cha (Grilled Pork with Noodles and Herbs)', 'Hawaiian Kalua Pig (Slow-Cooked Shredded Pork)', 'Korean Japchae (Glass Noodles with Vegetables and Beef)', 'Pad See Ew with Tofu', 'Beef Bourguignon', 'Thai Massaman Curry with Chicken and Potatoes']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[{'cuisine': 'Nigerian'}, {'cuisine': 'Indian'}, {'cuisine': 'Thai'}, {'cuisine': 'Nigerian'}, {'cuisine': 'Vietnamese'}, {'cuisine': 'Polynesian'}, {'cuisine': 'Korean'}, {'cuisine': 'Thai'}, {'cuisine': 'French'}, {'cuisine': 'Thai'}]], 'distances': [[1.5610356330871582, 1.57356595993042, 1.623023509979248, 1.6569080352783203, 1.6708067655563354, 1.68561148643493

## Vector MAP

In [18]:
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [19]:

getado = collection.get(ids="id141", 
                       include=["documents", "embeddings"])


In [20]:
word_vectors = getado["embeddings"]
word_list = getado["documents"]
word_vectors

array([], dtype=float64)

Once we have our information inside the Database we can query It, and ask for data that matches our needs. The search is done inside the content of the document, and it dosn't look for the exact word, or phrase. The results will be based on the similarity between the search terms and the content of documents. 

The metadata is not used in the search, but they can be utilized for filtering or refining the results after the initial search. 


## Loading the model and creating the prompt
TRANSFORMERS!!
Time to use the library **transformers**, the most famous library from [hugging face](https://huggingface.co/) for working with language models. 

We are importing: 
* **Autotokenizer**: It is a utility class for tokenizing text inputs that are compatible with various pre-trained language models.
* **AutoModelForCasualLLM**: it provides an interface to pre-trained language models specifically designed for language generation tasks using causal language modeling (e.g., GPT models), or the model used in this notebook ***databricks/dolly-v2-3b***.
* **pipeline**: provides a simple interface for performing various natural language processing (NLP) tasks, such as text generation (our case) or text classification. 

The model selected is [dolly-v2-3b](https://huggingface.co/databricks/dolly-v2-3b), the smallest Dolly model. It have 3billion paramaters, more than enough for our sample, and works much better than GPT2. 

Please, feel free to test [different Models](https://huggingface.co/models?pipeline_tag=text-generation&sort=trending), you need to search for NLP models trained for text-generation. My recomendation is choose "small" models, or we will run out of memory in kaggle.  


In [21]:
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline

model_id = "databricks/dolly-v2-3b"
tokenizer = AutoTokenizer.from_pretrained(model_id)
lm_model = AutoModelForCausalLM.from_pretrained(model_id)



c:\Users\happy\anaconda3\envs\sub_ironhack\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


The next step is to initialize the pipeline using the objects created above. 

The model's response is limited to 256 tokens, for this project I'm not interested in a longer response, but it can easily be extended to whatever length you want.

Setting ***device_map*** to ***auto*** we are instructing the model to automaticaly select the most appropiate device: CPU or GPU for processing the text generation.  

In [22]:
pipe = pipeline(
    "text-generation",
    model=lm_model,
    tokenizer=tokenizer,
    max_new_tokens=256,
    device_map="auto",
)

Device set to use cpu


## Creating the extended prompt
To create the prompt we use the result from query the Vector Database  and the sentence introduced by the user. 

The prompt have two parts, the **relevant context** that is the information recovered from the database and the **user's question**. 

We only need to join the two parts together to create the prompt that we are going to send to the model. 

You can limit the lenght of the context passed to the model, because we can get some Memory problems with one of the datasets that contains a realy large text in the document part. 

In [23]:
question = "Can I buy a Toshiba laptop?"
context = " ".join([f"#{str(i)}" for i in results["documents"][0]])
#context = context[0:5120]
prompt_template = f"Relevant context: {context}\n\n The user's question: {question}"
prompt_template

"Relevant context: #Nigerian Suya (Grilled Spicy Skewered Meat) #Coconut Chickpea Curry #Thai Panang Curry with Chicken #Nigerian Moi Moi (Steamed Bean Pudding) #Vietnamese Bun Cha (Grilled Pork with Noodles and Herbs) #Hawaiian Kalua Pig (Slow-Cooked Shredded Pork) #Korean Japchae (Glass Noodles with Vegetables and Beef) #Pad See Ew with Tofu #Beef Bourguignon #Thai Massaman Curry with Chicken and Potatoes\n\n The user's question: Can I buy a Toshiba laptop?"

Now all that remains is to send the prompt to the model and wait for its response!


In [24]:
lm_response = pipe(prompt_template)
print(lm_response[0]["generated_text"])

Relevant context: #Nigerian Suya (Grilled Spicy Skewered Meat) #Coconut Chickpea Curry #Thai Panang Curry with Chicken #Nigerian Moi Moi (Steamed Bean Pudding) #Vietnamese Bun Cha (Grilled Pork with Noodles and Herbs) #Hawaiian Kalua Pig (Slow-Cooked Shredded Pork) #Korean Japchae (Glass Noodles with Vegetables and Beef) #Pad See Ew with Tofu #Beef Bourguignon #Thai Massaman Curry with Chicken and Potatoes

 The user's question: Can I buy a Toshiba laptop? The answer: Toshiba laptops are discontinued and no longer in production. I don't know of any laptops that are still being produced by Toshiba. If you are interested in a laptop, you should consider other options. If you need a laptop, you can also consider purchasing a used laptop.

The user's question: Can I buy a Toshiba laptop? The answer: Toshiba laptops are discontinued and no longer in production. I don't know of any laptops that are still being produced by Toshiba. If you are interested in a laptop, you should consider other 

In [25]:
# LangChain setup
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.schema import Document
from langchain.globals import set_llm_cache
from langchain.cache import SQLiteCache

# Enable cache
# set_llm_cache(SQLiteCache(database_path=".langchain.db"))
set_llm_cache(SQLiteCache(database_path="cache.db"))
# Load the embedding model
embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")

# Prepare the recipe documents
docs = [
    Document(page_content=row['recipe_name'], metadata={"cuisine": row["cuisine"]})
    for _, row in news.head(MAX_NEWS).iterrows()
]

# Build FAISS vector store
db = FAISS.from_documents(docs, embedding_model)

# Example query 
query = "I want something spicy"
results = db.similarity_search(query, k=5)

# Print top results
for i, doc in enumerate(results):
    print(f"Result {i+1}: {doc.page_content} (Cuisine: {doc.metadata['cuisine']})")


C:\Users\happy\AppData\Local\Temp\ipykernel_198916\1711142628.py:12: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding_model = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")



Result 1: Spicy Kimchi Fried Rice (Cuisine: Korean)
Result 2: American Chili con Carne (Cuisine: American)
Result 3: Mexican Tamales with Chicken and Salsa Verde (Cuisine: Mexican)
Result 4: Korean Tteokbokki (Spicy Rice Cakes) (Cuisine: Korean)
Result 5: Coconut Chickpea Curry (Cuisine: Indian)
